In [419]:
import pandas as pd
import numpy as np
import sys

In [421]:
# read purchasing data and store in pandas dataframe 
df = pd.read_csv("purchase_data.csv")
df.head()
df[["SN"]].count()

SN    780
dtype: int64

In [422]:
# PLAYER COUNT

# create variable to calculate and hold total number of players 
df_players = df[["SN"]].drop_duplicates()
df_players_count = len(df_players)



# Create dataframe for display Total Players
total_players_df = pd.DataFrame({"Total Players" : [df_players_count]})
total_players_df

,Total Players
0,576


In [423]:
# calculate the average price , number of purchases, total revenue and unique item count

average_price = df["Price"].mean()
average_price

3.050987179487176

In [424]:
number_of_purchases = df["Purchase ID"].count()
number_of_purchases

780

In [425]:
total_revenue = df['Price'].sum()
total_revenue

2379.77

In [426]:
unique_items = df.groupby("Item ID")["Item Name"].unique()
unique = unique_items.count()

In [427]:
# PURCHASE ANALYSIS (Total)

# create dict for dataframe, Purchase Analysis
data = pd.DataFrame({"Number of Unique Items" : [unique], 
                     "Average Price" : [average_price], 
                     "Number of Purchases" : [number_of_purchases], 
                     "Total Revenue" : [total_revenue]
                    })

purchase_analysis_df = data


# use map to format all columns
purchase_analysis_df["Average Price"] = purchase_analysis_df["Average Price"].map("${:.2f}".format)
purchase_analysis_df["Total Revenue"] = purchase_analysis_df["Total Revenue"].map("${:.2f}".format)
purchase_analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


In [428]:
# PURCHASE ANALYSIS (Gender)

# ensure working with dataset that does not include duplicated screen names
df_drop = df.drop_duplicates("SN")


# variables to calculate and hold gender count values
male_count = df_drop.loc[df.Gender == 'Male', 'Gender'].count()
female_count = df_drop.loc[df.Gender == 'Female', 'Gender'].count()
other_count = df_drop.loc[df.Gender == 'Other / Non-Disclosed', 'Gender'].count()
total = male_count + female_count + other_count

In [429]:
#gender percentage values 
male_perc = (male_count / total)*100
female_perc = (female_count / total)*100
other_perc = (other_count / total)*100

male_perc = round(male_perc, 2)
female_perc = round(female_perc, 2)
other_perc = round(other_perc, 2)


In [430]:
# GENDER DEMOGRAPHICS SUMMARY

# create gender dataframe with dict[key:value] pairs
gender_df = {'Total Count' : pd.Series([male_count, female_count, other_count], index =['Male', 'Female', 'Other / Non-Disclosed']),
   'Total Percentage' : pd.Series([male_perc, female_perc, other_perc], index=['Male', 'Female', 'Other / Non-Disclosed'])}

gender_analysis_df = pd.DataFrame(gender_df)
gender_analysis_df

,Total Count,Total Percentage
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


In [431]:
# variables and calculations for the gender analysis dataframe
# cells 1-3 = female, male, other gender categories

female = df_drop[df_drop["Gender"]== "Female"].set_index("Gender")
avg_tot_f_pp = (female["Price"].sum()) / (female["SN"].nunique())
avg_f_pp = female.Price.mean()

In [432]:
male = df_drop[df_drop["Gender"]== "Male"]
avg_tot_m_pp = (male["Price"].sum()) / (male["SN"].nunique())
avg_m_pp = male.Price.mean()

In [433]:
other = df_drop[df_drop["Gender"]== "Other / Non-Disclosed"]
avg_tot_o_pp = (other["Price"].sum()) / (other["SN"].nunique())
avg_o_pp = other.Price.mean()

In [434]:
# create a dict to hold calculations of gender purchase summary info
gender_purchase = ({"Purchase Count" : [female["Purchase ID"].count(), male["Purchase ID"].count(), other["Purchase ID"].count()],
                            "Average Purchase Price" : [avg_f_pp, avg_m_pp, avg_o_pp],
                            "Total Purchase Value" : [female["Price"].sum(), male["Price"].sum(), other["Price"].sum()], 
                            "Average Total Purchase per Person" : [avg_tot_f_pp, avg_tot_m_pp, avg_tot_o_pp]
                   })


# create the dataframe
gender_purchase_summary = pd.DataFrame(gender_purchase, index = ["Female", "Male", "Other / Non-Disclosed"])


# using map, apply formatting to dataframe
gender_purchase_summary["Average Purchase Price"] = gender_purchase_summary["Average Purchase Price"].map("${:.2f}".format)
gender_purchase_summary["Total Purchase Value"] = gender_purchase_summary["Total Purchase Value"].map("${:.2f}".format)
gender_purchase_summary["Average Total Purchase per Person"] = gender_purchase_summary["Average Total Purchase per Person"].map("${:.2f}".format)
gender_purchase_summary 

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Female,81,$3.17,$256.43,$3.17
Male,484,$3.05,$1474.70,$3.05
Other / Non-Disclosed,11,$3.41,$37.52,$3.41


In [452]:
# create bins in which the data will be held
bins = [0, 9, 14, 19, 24, 29, 34.999, 39.999, 200]


# create labels for the respective bins
label_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

In [453]:
# AGE DEMOGRAPHICS SUMMARY


# create a new column in the dataset - list of corresponding age categories
df_drop["Total Count"] = pd.cut(df_drop["Age"], bins = bins, labels = label_names)
tally_totals = df_drop["Total Count"].value_counts().sort_index()



# convert to a dataframe & add the percentage of players column
age_demographic_df = pd.DataFrame(tally_totals)
age_demographic_df["Percentage of Players"] = round(((age_demographic_df.iloc[0:8:1] / 576)*100),2)
age_demographic_df

/Users/owner/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Total Count,Percentage of Players
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


In [454]:
# create a new column in the dataset that bins all of the data, including duplicated screen names
df["Age Groups"] = pd.cut(df["Age"], bins = bins, labels = label_names)

In [455]:

count_purchase = df.groupby(["Age Groups"])["Purchase ID"].count()
count_purchase

Age Groups
<10       23
10-14     28
15-19    136
20-24    365
25-29    101
30-34     73
35-39     41
40+       13
Name: Purchase ID, dtype: int64

In [456]:
# calculate variables for operations - use groupby 

count_purchase = df.groupby(["Age Groups"])["Purchase ID"].count()
avg_price = df.groupby(["Age Groups"])["Price"].mean()
purchase_total = df.groupby(["Age Groups"])["Price"].sum() 
avg_total_pp = df.groupby(["Age Groups"])["Price"].sum()
avg_total_pp = purchase_total / tally_totals

In [461]:
# PURCHASING ANALYSIS (age)

# create the summary dataframe - purchasing(age)
age_purchase_summary = pd.DataFrame({"Purchase Count": count_purchase, 
                                     "Average Purchase Price": avg_price, 
                                     "Total Purchase Value" : purchase_total, 
                                     "Average Total Purchase per Person" : avg_total_pp})


# using map, apply formatting to dataframe
age_purchase_summary["Average Purchase Price"] = age_purchase_summary["Average Purchase Price"].map("${:.2f}".format)
age_purchase_summary["Total Purchase Value"] = age_purchase_summary["Total Purchase Value"].map("${:.2f}".format)
age_purchase_summary["Average Total Purchase per Person"] = age_purchase_summary["Average Total Purchase per Person"].map("${:.2f}".format)


age_purchase_summary.sort_index(ascending = True)

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Age Groups,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [462]:
# TOPSPENDERS
# create variables to hold calculations
purchase_count = df.groupby(["SN"]).count()["Price"]
avg_purchase_price = df.groupby(["SN"]).mean()["Price"]
total_purchase_value = df.groupby(["SN"]).sum()["Price"]


# create the dataframe
data = pd.DataFrame({"Purchase Count": purchase_count,  "Average Purchase Price": avg_purchase_price, "Total Purchase Value": total_purchase_value})  

# sort the dataframe by total purchase volume
top_spenders = data.sort_values("Total Purchase Value", ascending = False)
top_spenders["Average Purchase Price"] = top_spenders["Average Purchase Price"].map("${:.2f}".format)


#top_spender_df = pd.DataFrame.from_dict(top_spenders)
top_spenders.head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,18.96
Idastidru52,4,$3.86,15.45
Chamjask73,3,$4.61,13.83
Iral74,4,$3.40,13.62
Iskadarya95,3,$4.37,13.10


In [463]:
# POPULAR ITEMS SUMMARY

pop_items = df.loc[:,["Item ID", "Item Name", "Price"]]

# create variables to hold calculations
purchase_count = pop_items.groupby(["Item ID", "Item Name"]).count()["Price"]
item_price = pop_items.groupby(["Item ID", "Item Name"]).mean()["Price"]
total_purchase_value = pop_items.groupby(["Item ID", "Item Name"]).sum()["Price"]


# create the dataframe
data = pd.DataFrame({"Purchase Count": purchase_count,  "Average Purchase Price": item_price, "Total Purchase Value": total_purchase_value})  # sort the dataframe by total purchase volume
top_items = data.sort_values("Purchase Count", ascending = False)
top_items_df = pd.DataFrame.from_dict(top_items)

top_items_df.head(5)

,,Purchase Count,Average Purchase Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
145,Fiery Glass Crusader,9,4.58,41.22
108,"Extraction, Quickblade Of Trembling Hands",9,3.53,31.77
82,Nirvana,9,4.90,44.10
19,"Pursuit, Cudgel of Necromancy",8,1.02,8.16


In [464]:

# sort the above dataframe by total purchase volume, descending order
profit_items = top_items_df.sort_values("Total Purchase Value", ascending = False)
profit_items_df = pd.DataFrame(profit_items)

profit_items_df.head(5)

,,Purchase Count,Average Purchase Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
82,Nirvana,9,4.90,44.10
145,Fiery Glass Crusader,9,4.58,41.22
92,Final Critic,8,4.88,39.04
103,Singed Scalpel,8,4.35,34.80
